<a href="https://colab.research.google.com/github/sanjay-sj87/gen-AI/blob/main/Basic_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install faiss-cpu -q

In [8]:
!pip uninstall -y torch torchvision torchaudio
!pip install torch --index-url https://download.pytorch.org/whl/cpu
!pip install sentence-transformers transformers faiss-cpu


Found existing installation: torch 2.9.0+cpu
Uninstalling torch-2.9.0+cpu:
  Successfully uninstalled torch-2.9.0+cpu
Found existing installation: torchvision 0.24.0+cpu
Uninstalling torchvision-0.24.0+cpu:
  Successfully uninstalled torchvision-0.24.0+cpu
Found existing installation: torchaudio 2.9.0+cpu
Uninstalling torchaudio-2.9.0+cpu:
  Successfully uninstalled torchaudio-2.9.0+cpu
Looking in indexes: https://download.pytorch.org/whl/cpu
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.9/188.9 MB 7.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 2.8.6 requires torchvision>=0.11, which is not installed.
timm 1.0.24 requires torchvision, which is not installed.
fastai 2.8.6 requires torch<2.10,>=1.10, but you have torch 2.10.0+cpu which is incompatible.


Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
^C


In [2]:
from sentence_transformers import SentenceTransformer


In [3]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

In [4]:
import numpy as np

In [5]:
import faiss

In [6]:
import os, getpass

In [7]:
os.environ['HUGGINGFACE_TOKEN'] = getpass.getpass('Huggingface Token:')

Huggingface Token:··········


In [8]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/148 [00:00<?, ?it/s]

GPT2LMHeadModel LOAD REPORT from: gpt2
Key                  | Status     |  | 
---------------------+------------+--+-
h.{0...11}.attn.bias | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [9]:
tokenizer.pad_token = tokenizer.eos_token

In [10]:
def get_embeddings(texts):
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True)
    # print(inputs.keys())
    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True)
    embeddings = outputs.hidden_states[-1][:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)

    return embeddings

In [11]:
# retriever_model = SentenceTransformer('bert-base-nli-mean-tokens')

In [12]:
documents = [
    "A Christmas Carol is a novella by Charles Dickens, first published in London on 19 December 1843.",
    "The story tells of sour and stingy Ebenezer Scrooge's ideological, ethical, and emotional transformation after \n"
    "the supernatural visits of Jacob Marley and the Ghosts of Christmas Past, Present, and Yet to Come.",
    "The novella met with instant success and critical acclaim. It is regarded as one of the greatest Christmas stories ever written."
]
query = "What is the significance of Christmas Eve in A Christmas Carol?"

In [13]:
query_embedding = get_embeddings([query])
document_embeddings = get_embeddings(documents)

In [14]:
document_embeddings.shape

torch.Size([3, 768])

In [15]:
# def cosine_similarity(a, b):
#     return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def cosine_similarity(embedding1, embedding2):
    return torch.nn.functional.cosine_similarity(embedding1, embedding2)


In [16]:
for doc_embedding in document_embeddings:
  print(cosine_similarity(query_embedding, doc_embedding))

tensor([0.9757])
tensor([0.9959])
tensor([0.9959])


In [17]:
similarities = [cosine_similarity(query_embedding, doc_embedding) for doc_embedding in document_embeddings]

In [18]:
similarities

[tensor([0.9757]), tensor([0.9959]), tensor([0.9959])]

In [19]:
# documents

In [20]:
ranked_documents = sorted(zip(documents, similarities), key=lambda x: x[1], reverse=True)

In [21]:
ranked_documents

[("The story tells of sour and stingy Ebenezer Scrooge's ideological, ethical, and emotional transformation after \nthe supernatural visits of Jacob Marley and the Ghosts of Christmas Past, Present, and Yet to Come.",
  tensor([0.9959])),
 ('The novella met with instant success and critical acclaim. It is regarded as one of the greatest Christmas stories ever written.',
  tensor([0.9959])),
 ('A Christmas Carol is a novella by Charles Dickens, first published in London on 19 December 1843.',
  tensor([0.9757]))]

In [22]:
top_documents = [doc for doc, _ in ranked_documents[:2]]
print(top_documents)

["The story tells of sour and stingy Ebenezer Scrooge's ideological, ethical, and emotional transformation after \nthe supernatural visits of Jacob Marley and the Ghosts of Christmas Past, Present, and Yet to Come.", 'The novella met with instant success and critical acclaim. It is regarded as one of the greatest Christmas stories ever written.']


In [23]:
# Generation

In [24]:
query + " [SEP] " + " ".join(top_documents)

"What is the significance of Christmas Eve in A Christmas Carol? [SEP] The story tells of sour and stingy Ebenezer Scrooge's ideological, ethical, and emotional transformation after \nthe supernatural visits of Jacob Marley and the Ghosts of Christmas Past, Present, and Yet to Come. The novella met with instant success and critical acclaim. It is regarded as one of the greatest Christmas stories ever written."

In [25]:
augmented_input = query + " [SEP] " + " ".join(top_documents)

In [26]:
input_ids = tokenizer.encode(augmented_input, return_tensors="pt", padding=True, truncation=True)

In [27]:
input_ids

tensor([[ 2061,   318,   262, 12085,   286,  6786, 12882,   287,   317,  6786,
          5074,    30,   685,  5188,    47,    60,   383,  1621,  4952,   286,
         11348,   290, 24276,    88, 12119,  1734,  9107,  1446, 42407,   469,
           338, 15735,    11, 15028,    11,   290,  7016, 13389,   706,   220,
           198,  1169, 22239, 11864,   286, 12806,  1526,  1636,   290,   262,
         38389,   286,  6786, 11303,    11, 21662,    11,   290,  6430,   284,
          7911,    13,   383,   645,   303,  8466,  1138,   351,  9113,  1943,
           290,  4688, 21684,    13,   632,   318, 11987,   355,   530,   286,
           262,  6000,  6786,  3923,  1683,  3194,    13]])

In [28]:
outputs = model.generate(input_ids, max_length=150, num_beams=2, early_stopping=True)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [29]:
outputs

tensor([[ 2061,   318,   262, 12085,   286,  6786, 12882,   287,   317,  6786,
          5074,    30,   685,  5188,    47,    60,   383,  1621,  4952,   286,
         11348,   290, 24276,    88, 12119,  1734,  9107,  1446, 42407,   469,
           338, 15735,    11, 15028,    11,   290,  7016, 13389,   706,   220,
           198,  1169, 22239, 11864,   286, 12806,  1526,  1636,   290,   262,
         38389,   286,  6786, 11303,    11, 21662,    11,   290,  6430,   284,
          7911,    13,   383,   645,   303,  8466,  1138,   351,  9113,  1943,
           290,  4688, 21684,    13,   632,   318, 11987,   355,   530,   286,
           262,  6000,  6786,  3923,  1683,  3194,    13,   198,   198,    58,
          5188,    47,    60,   383,  1621,  4952,   286, 11348,   290, 24276,
            88, 12119,  1734,  9107,  1446, 42407,   469,   338, 15735,    11,
         15028,    11,   290,  7016, 13389,   706,   262, 22239, 11864,   286,
         12806,  1526,  1636,   290,   262, 38389,  

In [30]:
generated_response = tokenizer.decode(outputs[0], skip_special_tokens=True)

In [31]:
print(generated_response)

What is the significance of Christmas Eve in A Christmas Carol? [SEP] The story tells of sour and stingy Ebenezer Scrooge's ideological, ethical, and emotional transformation after 
the supernatural visits of Jacob Marley and the Ghosts of Christmas Past, Present, and Yet to Come. The novella met with instant success and critical acclaim. It is regarded as one of the greatest Christmas stories ever written.

[SEP] The story tells of sour and stingy Ebenezer Scrooge's ideological, ethical, and emotional transformation after the supernatural visits of Jacob Marley and the Ghosts of Christmas Past, Present, and Yet to Come. The novella met with instant success and critical acclaim. It


**With Vectordb**

In [32]:
document_embeddings.shape

torch.Size([3, 768])

In [33]:
# Create FAISS index
index = faiss.IndexFlatL2(document_embeddings.shape[1])

In [34]:
index.is_trained

True

In [35]:
index.add(document_embeddings.numpy())

In [36]:
index.ntotal

3

In [37]:
# Retrieve information
query_embedding = get_embeddings([query])

In [38]:
distances, indices = index.search(query_embedding.detach().numpy(), k=5)

In [39]:
distances[0], indices[0]

(array([8.2615372e-03, 8.2615372e-03, 4.8689429e-02, 3.4028235e+38,
        3.4028235e+38], dtype=float32),
 array([ 1,  2,  0, -1, -1]))

In [40]:
# Get top documents
top_documents = [documents[i] for i in indices[0]]

In [41]:
top_documents

["The story tells of sour and stingy Ebenezer Scrooge's ideological, ethical, and emotional transformation after \nthe supernatural visits of Jacob Marley and the Ghosts of Christmas Past, Present, and Yet to Come.",
 'The novella met with instant success and critical acclaim. It is regarded as one of the greatest Christmas stories ever written.',
 'A Christmas Carol is a novella by Charles Dickens, first published in London on 19 December 1843.',
 'The novella met with instant success and critical acclaim. It is regarded as one of the greatest Christmas stories ever written.',
 'The novella met with instant success and critical acclaim. It is regarded as one of the greatest Christmas stories ever written.']

In [42]:
augmented_input = query + " [SEP] " + " ".join(top_documents)

In [43]:
# Generate the response
input_ids = tokenizer.encode(augmented_input, return_tensors="pt", padding=True, truncation=True)

In [44]:
input_ids

tensor([[ 2061,   318,   262, 12085,   286,  6786, 12882,   287,   317,  6786,
          5074,    30,   685,  5188,    47,    60,   383,  1621,  4952,   286,
         11348,   290, 24276,    88, 12119,  1734,  9107,  1446, 42407,   469,
           338, 15735,    11, 15028,    11,   290,  7016, 13389,   706,   220,
           198,  1169, 22239, 11864,   286, 12806,  1526,  1636,   290,   262,
         38389,   286,  6786, 11303,    11, 21662,    11,   290,  6430,   284,
          7911,    13,   383,   645,   303,  8466,  1138,   351,  9113,  1943,
           290,  4688, 21684,    13,   632,   318, 11987,   355,   530,   286,
           262,  6000,  6786,  3923,  1683,  3194,    13,   317,  6786,  5074,
           318,   257,   645,   303,  8466,   416,  7516, 46167,    11,   717,
          3199,   287,  3576,   319,   678,  3426,  1248,  3559,    13,   383,
           645,   303,  8466,  1138,   351,  9113,  1943,   290,  4688, 21684,
            13,   632,   318, 11987,   355,   530,  

In [45]:
input_ids

tensor([[ 2061,   318,   262, 12085,   286,  6786, 12882,   287,   317,  6786,
          5074,    30,   685,  5188,    47,    60,   383,  1621,  4952,   286,
         11348,   290, 24276,    88, 12119,  1734,  9107,  1446, 42407,   469,
           338, 15735,    11, 15028,    11,   290,  7016, 13389,   706,   220,
           198,  1169, 22239, 11864,   286, 12806,  1526,  1636,   290,   262,
         38389,   286,  6786, 11303,    11, 21662,    11,   290,  6430,   284,
          7911,    13,   383,   645,   303,  8466,  1138,   351,  9113,  1943,
           290,  4688, 21684,    13,   632,   318, 11987,   355,   530,   286,
           262,  6000,  6786,  3923,  1683,  3194,    13,   317,  6786,  5074,
           318,   257,   645,   303,  8466,   416,  7516, 46167,    11,   717,
          3199,   287,  3576,   319,   678,  3426,  1248,  3559,    13,   383,
           645,   303,  8466,  1138,   351,  9113,  1943,   290,  4688, 21684,
            13,   632,   318, 11987,   355,   530,  

In [46]:
outputs = model.generate(input_ids, max_length=160, num_beams=2, early_stopping=True)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [47]:
generated_response = tokenizer.decode(outputs[0], skip_special_tokens=True)

In [48]:
generated_response

"What is the significance of Christmas Eve in A Christmas Carol? [SEP] The story tells of sour and stingy Ebenezer Scrooge's ideological, ethical, and emotional transformation after \nthe supernatural visits of Jacob Marley and the Ghosts of Christmas Past, Present, and Yet to Come. The novella met with instant success and critical acclaim. It is regarded as one of the greatest Christmas stories ever written. A Christmas Carol is a novella by Charles Dickens, first published in London on 19 December 1843. The novella met with instant success and critical acclaim. It is regarded as one of the greatest Christmas stories ever written. The novella met with instant success and critical acclaim. It is regarded as one of the greatest Christmas stories ever written.\n"

**With Chunks of Data**

In [49]:
# embeddings
emb = np.random.random((10, 128)).astype('float32')

In [50]:
emb.shape

(10, 128)

In [51]:
index = faiss.IndexFlatL2(128)

In [52]:
index.add(emb)

In [53]:
# Query
quer = np.random.random((1, 128)).astype('float32')

In [54]:
# search
distances, indices = index.search(quer, k=13)

In [55]:
distances

array([[1.7067965e+01, 1.7112309e+01, 1.7996447e+01, 1.8208414e+01,
        1.8888557e+01, 1.9032751e+01, 1.9303852e+01, 1.9350075e+01,
        2.1702934e+01, 2.2326180e+01, 3.4028235e+38, 3.4028235e+38,
        3.4028235e+38]], dtype=float32)

In [56]:
indices

array([[ 2,  1,  0,  4,  3,  5,  9,  8,  6,  7, -1, -1, -1]])

In [57]:
stdoc = "The story tells of sour and stingy Ebenezer Scrooge's ideological, ethical, and emotional transformation after the supernatural visits of Jacob Marley and the Ghosts of Christmas Past, Present, and Yet to Come."

In [58]:
w = stdoc.split()

In [59]:
for i in range(0, len(w), 5):
  print(' '.join(w[i:i+5]))

The story tells of sour
and stingy Ebenezer Scrooge's ideological,
ethical, and emotional transformation after
the supernatural visits of Jacob
Marley and the Ghosts of
Christmas Past, Present, and Yet
to Come.


In [60]:
# Function to chunk text
def chunk_text(text, max_length=100):
    words = text.split()
    chunks = [' '.join(words[i:i + max_length]) for i in range(0, len(words), max_length)]
    return chunks

In [61]:
chunks = []
for doc in documents:
    chunks.extend(chunk_text(doc, max_length=21))  # Adjust max_length as needed


In [62]:
chunks

['A Christmas Carol is a novella by Charles Dickens, first published in London on 19 December 1843.',
 "The story tells of sour and stingy Ebenezer Scrooge's ideological, ethical, and emotional transformation after the supernatural visits of Jacob Marley",
 'and the Ghosts of Christmas Past, Present, and Yet to Come.',
 'The novella met with instant success and critical acclaim. It is regarded as one of the greatest Christmas stories ever written.']

In [63]:
# Get chunk embeddings
chunk_embeddings = get_embeddings(chunks)

In [64]:
index = faiss.IndexFlatL2(chunk_embeddings.shape[1])


In [65]:
index.add(chunk_embeddings.detach().numpy())

In [66]:
query_embedding = get_embeddings([query])
distances, indices = index.search(query_embedding.detach().numpy(), k=7)

In [67]:
indices

array([[ 1,  3,  2,  0, -1, -1, -1]])

In [68]:
# Get top chunks
top_chunks = [chunks[i] for i in indices[0]]
augmented_input = query + " [SEP] " + " ".join(top_chunks)
print("Augmented Input:", augmented_input)

Augmented Input: What is the significance of Christmas Eve in A Christmas Carol? [SEP] The story tells of sour and stingy Ebenezer Scrooge's ideological, ethical, and emotional transformation after the supernatural visits of Jacob Marley The novella met with instant success and critical acclaim. It is regarded as one of the greatest Christmas stories ever written. and the Ghosts of Christmas Past, Present, and Yet to Come. A Christmas Carol is a novella by Charles Dickens, first published in London on 19 December 1843. The novella met with instant success and critical acclaim. It is regarded as one of the greatest Christmas stories ever written. The novella met with instant success and critical acclaim. It is regarded as one of the greatest Christmas stories ever written. The novella met with instant success and critical acclaim. It is regarded as one of the greatest Christmas stories ever written.


In [69]:
input_ids = tokenizer.encode(augmented_input, return_tensors="pt", padding=True, truncation=True)

In [70]:
outputs = model.generate(input_ids, max_length=190, num_beams=2, early_stopping=True)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [71]:
out = tokenizer.decode(outputs[0], skip_special_tokens=True)


In [72]:
out.split('[SEP]')[0]

'What is the significance of Christmas Eve in A Christmas Carol? '

In [73]:
out.split('[SEP]')[1]

" The story tells of sour and stingy Ebenezer Scrooge's ideological, ethical, and emotional transformation after the supernatural visits of Jacob Marley The novella met with instant success and critical acclaim. It is regarded as one of the greatest Christmas stories ever written. and the Ghosts of Christmas Past, Present, and Yet to Come. A Christmas Carol is a novella by Charles Dickens, first published in London on 19 December 1843. The novella met with instant success and critical acclaim. It is regarded as one of the greatest Christmas stories ever written. The novella met with instant success and critical acclaim. It is regarded as one of the greatest Christmas stories ever written. The novella met with instant success and critical acclaim. It is regarded as one of the greatest Christmas stories ever written. The novella met with instant success"

In [74]:
# What is RAG, Why RAG, RAG Architecture, Why not fine tuning
# Update
# HyperLinks
# Diversified documents
# Metrics
# Hallucinations and wrong answers
# Updating the query
# Best practices
# RAG failure scenarios